In [ ]:
# read data

from mnist import MNIST
import numpy as np
mndata = MNIST("/Users/jigyayadav/Desktop/Codes/neuralnets253/HW1")
mndata.gz = True
images, labels = mndata.load_training() #Images is a list of 60000 images of 784 dimensions, Labels is a list of 60000 ints
imagesTest, labelsTest = mndata.load_testing()

images = np.array(images)
labels = np.array(labels)
imagesTest = np.array(imagesTest)
labelsTest = np.array(labelsTest)
images = np.insert(images, 0, 1, axis=1)
imagesTest = np.insert(imagesTest, 0, 1, axis=1)

In [ ]:
images_train_all = images[:20000]
images_test_all = imagesTest[-2000:]
labels_train_all = labels[:20000]
labels_test_all = labelsTest[-2000:]

images_train = []
images_test = []
labels_train = []
labels_test = []

# Extract the training examples for 2 and 3
for i in range(len(labels_train_all)):
    if (labels_train_all[i] == 2):
        images_train.append(images_train_all[i])
        labels_train.append(1)
    if (labels_train_all[i] == 3):
        images_train.append(images_train_all[i])
        labels_train.append(0)
        
for i in range(len(labels_test_all)):
    if (labels_test_all[i] == 2):
        images_test.append(images_test_all[i])
        labels_test.append(1)
    if (labels_test_all[i] == 3):
        images_test.append(images_test_all[i])
        labels_test.append(0)

images_train = np.array(images_train)
labels_train = np.array(labels_train)
images_test = np.array(images_test)
labels_test = np.array(labels_test)

# Normalization
images_train = images_train*(1/255)
images_test = images_test*(1/255)

numFeatures = len(images[0, :])

# Divide between validation and training
from sklearn.model_selection import train_test_split
images_train, images_validation, labels_train, labels_validation = train_test_split(images_train, labels_train, test_size=0.10)

In [ ]:
# Helper functions

import math
from sklearn.metrics import accuracy_score

numIterations = 10000

# Predict label based on the pr
def predictLabel(a):
    if (a >= 0.5):
        return 1
    return 0

def predictionAccuracy(x, weights, t):
    predictedOutput = predictY(x, weights)
    for i in range(len(predictedOutput)):
        predictedOutput[i] = predictLabel(predictedOutput[i])
    return accuracy_score(t, predictedOutput)

def errorFunction(weights, x, t, reg, l2):
    numExamples = len(x)
    predictedOutput = predictY(x, weights)
    error = 0
    examplesCounted = 0
    for i in range(numExamples):
        if (predictedOutput[i] < 1 and predictedOutput[i] > 0):
            error += t[i]*math.log(predictedOutput[i])+(1-t[i])*math.log(1-predictedOutput[i])
            examplesCounted += 1
    error = -1*(error/examplesCounted)
    if(l2):
        error += reg*np.sum(np.square(weights))
    else:
        error += reg*np.sum(np.abs(weights))
    return error

def sigmoid(x):
    return 1 / (1 + math.exp(-x))

def predictY(x, weights):
    predictedOutput = np.dot(x, weights)
    for i in range(len(predictedOutput)):
        predictedOutput[i] = sigmoid(predictedOutput[i])
    return predictedOutput

def updateWeights(weights, x, t, learningRate, regularization, l2):
    predicted = predictY(x, weights)
    diffPredictedActual = predicted-t
    differentiation = (diffPredictedActual.T).dot(x)
    differentiation = differentiation*(1/len(x))
    if (l2 == True):
        differentiation += 2*regularization*weights
    else:
        diffRegularizer = np.zeros(len(weights))
        for i in range(len(weights)):
            if weights[i] == 0:
                diffRegularizer[i] = 0
            elif weights[i] > 0:
                diffRegularizer[i] = 1
            else:
                diffRegularizer[i] = -1
        differentiation += regularization*(diffRegularizer)
    differentiation = differentiation
    newWeights = (weights-learningRate*differentiation)
    return newWeights

In [ ]:
import matplotlib.pyplot as plt

def gradientDescent(x, t, testLam, regularization, annealingParameter, l2):
    images_train = x[0]
    images_validation = x[1]
    images_test = x[2]
    labels_train = t[0]
    labels_validation = t[1]
    labels_test = t[2]
    
    weights = np.random.normal(0, 1, numFeatures)
#     weights = np.zeros(numFeatures)
    weightsMinError = np.zeros(numFeatures)
    minErrorFunction = float('inf')
    prevValidationError = 0.0
    cnt = 0
    stoppingIteration = 1
    iterationMinWeight = 0
    arrTrainingError = []
    arrValidationError = []
    arrTestError = []
    arrTrainClassificationAccuracy = []
    arrValidationClassificationAccuracy = []
    arrTestClassificationAccuracy = []

    for i in range(10000):
        currLearningRate = testLam/(1+(i/annealingParameter))
        weights = updateWeights(weights, images_train, labels_train, currLearningRate, regularization, l2)
        trainAccuracy = predictionAccuracy(images_train, weights, labels_train)
        validationAccuracy = predictionAccuracy(images_validation, weights, labels_validation)
        testAccuracy = predictionAccuracy(images_test, weights, labels_test)
        currValidationError = errorFunction(weights, images_validation, labels_validation, regularization, l2)
        currTestError = errorFunction(weights, images_test, labels_test, regularization, l2)
        currTrainError = errorFunction(weights, images_train, labels_train, regularization, l2)
        arrTrainingError.append(currTrainError)
        arrValidationError.append(currValidationError)
        arrTestError.append(currTestError)
        arrTrainClassificationAccuracy.append(trainAccuracy)
        arrValidationClassificationAccuracy.append(validationAccuracy)
        arrTestClassificationAccuracy.append(testAccuracy)
        if (currValidationError < minErrorFunction):
            weightsMinError = weights
            minErrorFunction = currValidationError
            iterationMinWeight = i
        if (currValidationError > prevValidationError):
            cnt += 1
            if (cnt >= 3):
                stoppingIteration = i
                break
        else:
            cnt = 0
        prevValidationError = currValidationError

    testAccuracyFinal = predictionAccuracy(images_test, weightsMinError, labels_test)
    validationAccuracyFinal = predictionAccuracy(images_validation, weightsMinError, labels_validation)
    trainAccuracyFinal = predictionAccuracy(images_train, weightsMinError, labels_train)
    print("Final train classification accuracy = ", trainAccuracyFinal)
    print("Final validation classification accuracy = ", validationAccuracyFinal)
    print("Final test classification accuracy = ", testAccuracyFinal)

    plt.figure(1)
    plt.subplot(211)
    plt.plot(arrTrainingError, label='Train', color='blue')
    plt.plot(arrValidationError, label='Validation', color='green')
    plt.plot(arrTestError, label='Test', color='red')
    plt.ylabel('Error')
    plt.xlabel('Iterations')
    plt.legend()

    plt.subplot(212)
    plt.plot(arrTrainClassificationAccuracy, label='Train', color='blue')
    plt.plot(arrValidationClassificationAccuracy, label='Validation', color='green')
    plt.plot(arrTestClassificationAccuracy, label='Test', color='red')
    plt.ylabel('Accuracy')
    plt.xlabel('Iterations')
    plt.legend()
    plt.show()
    
    testErrorFinal = errorFunction(weightsMinError, images_test, labels_test, l2, regularization)
    
    return [weightsMinError, testErrorFinal, testAccuracyFinal, arrTrainClassificationAccuracy]
    
    print("********************************************************************************")

In [ ]:
# Figuring out best annealing parameter and learning rate

maxValidationClassificationAcc = float("-inf")
bestLearningRate = 0.001
bestAnnealingParam = 10000

for testLam in [0.1, 0.01, 0.001, 0.0001]:
    for annealingParameter in [5000, 10000, 20000]:
        learningRate = testLam
        weights = np.random.normal(0, 1, numFeatures)
        weightsWhichMinimizeError = np.zeros(numFeatures)
        minErrorFunction = float('inf')
        prevValidationError = 0.0
        cnt = 0
        stoppingIteration = 1
        iterationMinWeight = 0
        for i in range(10000):
            learningRate = testLam/(1+i/annealingParameter)
            weights = updateWeights(weights, images_train, labels_train)
            currValidationError = errorFunction(weights, images_validation, labels_validation, 0.0, True)
            if (currValidationError < minErrorFunction):
                weightsWhichMinimizeError = weights
                minErrorFunction = currValidationError
                iterationMinWeight = i
            if (currValidationError > prevValidationError):
                cnt += 1
                if (cnt >= 3):
                    stoppingIteration = i
                    break
            else:
                cnt = 0
            prevValidationError = currValidationError

        validationAccuracyFinal = predictionAccuracy(images_validation, weightsWhichMinimizeError, labels_validation)
        print("Final validation classification accuracy = ", validationAccuracyFinal)
        if (validationAccuracyFinal > maxValidationClassificationAcc):
            maxValidationClassificationAcc = validationAccuracyFinal
            bestLearningRate = testLam
            bestAnnealingParam = annealingParameter

In [ ]:
# Best learning rate = 0.1
# No regularization

x = [images_train, images_validation, images_test]
t = [labels_train, labels_validation, labels_test]
weightsCurr = gradientDescent(x, t, 0.1, 0.0, 10000, True)

weightsImage = np.resize(np.array(weightsCurr[0][1:]),(28,28))
plt.imshow(weightsImage,interpolation='none')
plt.title("Weights for 2-3 digit classification")
plt.show()

In [ ]:
# 2 vs 8

images_train_8 = []
images_test_8 = []
labels_train_8 = []
labels_test_8 = []

for i in range(len(labels_train_all)):
    if (labels_train_all[i] == 2):
        images_train_8.append(images_train_all[i])
        labels_train_8.append(1)
    if (labels_train_all[i] == 8):
        images_train_8.append(images_train_all[i])
        labels_train_8.append(0)
        
for i in range(len(labels_test_all)):
    if (labels_test_all[i] == 2):
        images_test_8.append(images_test_all[i])
        labels_test_8.append(1)
    if (labels_test_all[i] == 8):
        images_test_8.append(images_test_all[i])
        labels_test_8.append(0)

images_train_8 = np.array(images_train_8)
labels_train_8 = np.array(labels_train_8)
images_test_8 = np.array(images_test_8)
labels_test_8 = np.array(labels_test_8)

images_train_8 = images_train_8*(1/255)
images_test_8 = images_test_8*(1/255)

images_train_8, images_validation_8, labels_train_8, labels_validation_8 = train_test_split(images_train_8, labels_train_8, test_size=0.10)

x = [images_train_8, images_validation_8, images_test_8]
t = [labels_train_8, labels_validation_8, labels_test_8]
weights_8 = gradientDescent(x, t, 0.1, 0.0, 10000, True)

# Plot the weight for 2-8
weightsImage_8 = np.resize(np.array(weights_8[0][1:]),(28,28))
plt.imshow(weightsImage_8,interpolation='none')
plt.title("Weights for 2-8 digit classification")
plt.show()

In [ ]:
# Plot difference between weights
diff38 = weightsImage-weightsImage_8
plt.imshow(diff38,interpolation='none')
plt.title("Difference between weights for 2,3 and 2,8")
plt.show()

In [ ]:
# Test accuracy on 3 vs 8 using the difference of weights

weightsFor38 = weights_8[0]-weightsCurr[0]
images_test_3 = []
labels_test_3 = []
        
for i in range(len(labels_test_all)):
    if (labels_test_all[i] == 3):
        images_test_3.append(images_test_all[i])
        labels_test_3.append(1)
    if (labels_test_all[i] == 8):
        images_test_3.append(images_test_all[i])
        labels_test_3.append(0)

images_test_3 = np.array(images_test_3)
labels_test_3 = np.array(labels_test_3)

images_test_3 = images_test_3*(1/255)

predictionAccuracy(images_test_3, weightsFor38, labels_test_3)

In [ ]:
# With regularization L2

x = [images_train, images_validation, images_test]
t = [labels_train, labels_validation, labels_test]
weightsL2Regularization = []
testErrorsL2Regularization = []
testAccuracyL2Regularization = []
trainingAccuracyL2RegVecs = []

learning_rate = 0.1
for reg in [0.0001, 0.001, 0.01, 0.1, 1]:
    print("Regularization = ", reg, "Learning Rate = ", learning_rate)
    resultCurr = gradientDescent(x, t, learning_rate, reg, 10000, True)
    weightsL2Regularization.append(resultCurr[0])
    testErrorsL2Regularization.append(resultCurr[1])
    testAccuracyL2Regularization.append(resultCurr[2])
    trainingAccuracyL2RegVecs.append(resultCurr[3])

In [ ]:
# Plot training error classification over lambda

lambdaVec = [0.0001, 0.001, 0.01, 0.1, 1]
logLambdaVec = [np.log(i) for i in lambdaVec]

for i in range(len(trainingAccuracyL2RegVecs)):
    currLabel = lambdaVec[i]
    plt.plot(trainingAccuracyL2RegVecs[i], label=currLabel)
    plt.ylabel('Training Accuracy')
    plt.xlabel('Lambda')
plt.
plt.legend()
plt.show()

In [ ]:
import math

y_axis = [math.sqrt(np.sum(np.square(i))) for i in weightsL2Regularization]
plt.plot(lambdaVec, y_axis, linestyle='--', marker='o')
plt.ylabel('Length of weight vector')
plt.xlabel('Regularization')
plt.show()

In [ ]:
plt.plot(logLambdaVec, testErrorsL2Regularization)
plt.ylabel('Final test error')
plt.xlabel('Log of Regularization')
plt.show()

In [ ]:
weightsImageTemp = np.resize(np.array(weightsL2Regularization[0][1:]),(28,28))
plt.imshow(weightsImageTemp,interpolation='none')
plt.title("Weights for 2-3 image, Reg = 0.0001")
plt.show()

In [ ]:
# With regularization L1

weightsL1Regularization2 = []
testErrorsL1Regularization2 = []
testAccuracyL1Regularization2 = []
trainingAccuracyL1RegVecs2 = []

for reg in [0.0001, 0.001, 0.01, 0.1, 1, 10]:
    for learning_rate in [0.01]:
        print("Regularization = ", reg, "Learning Rate = ", learning_rate)
        resultCurr = gradientDescent(x, t, learning_rate, reg, 10000, False)
        weightsL1Regularization2.append(resultCurr[0])
        testErrorsL1Regularization2.append(resultCurr[1])
        testAccuracyL1Regularization2.append(resultCurr[2])
        trainingAccuracyL1RegVecs2.append(resultCurr[3])
        
        
lambdaVecL1_2 = [0.0001, 0.001, 0.01, 0.1, 1, 10]
logLambdaVecL1_2 = [np.log(i) for i in lambdaVecL1_2]

for i in range(len(trainingAccuracyL1RegVecs2)):
    currLabel = lambdaVecL1_2[i]
    plt.plot(trainingAccuracyL1RegVecs2[i], label=currLabel)
    plt.ylabel('Training Accuracy')
    plt.xlabel('Lambda')

plt.legend()
plt.show()

In [ ]:
y_axis_L1_2 = [math.sqrt(np.sum(np.square(i))) for i in weightsL1Regularization2]
plt.plot(lambdaVecL1_2, y_axis_L1_2, linestyle='--', marker='o')
plt.ylabel('Length of weight vector')
plt.xlabel('Regularization')
plt.show()

In [ ]:
plt.plot(logLambdaVecL1_2, testErrorsL1Regularization2, linestyle='--', marker='o')
plt.ylabel('Final test error')
plt.xlabel('Log of Regularization')
plt.show()